In [6]:
from datasets import load_from_disk

# dataset = load_from_disk("/appdataset/train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415")
# train_dataset = load_from_disk("Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_train_3.3M_0415")
# validation_dataset = load_from_disk("Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_validation_3.3M_0415")
test_dataset = load_from_disk("Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415")

# from collections import defaultdict
# from tqdm import tqdm

# task_cnt = defaultdict(int)

# for data in tqdm(test_dataset):
#     task_cnt[data['task']] += 1
    
# task_cnt

In [7]:
import pandas as pd

# Pandas Series로 변환 후 value_counts 실행
task_counts = pd.Series(test_dataset['task']).value_counts()

print(task_counts)

presto-retrosynthesis                 100
presto-forward_reaction_prediction    100
alchemy_homo                          100
aqsol-logS                            100
alchemy_homo_lumo_gap                 100
alchemy_lumo                          100
Name: count, dtype: int64


In [8]:
import random
from collections import defaultdict
from tqdm import tqdm

# 1. 각 Task별로 데이터가 어디에 있는지 인덱스(Index) 수집
# (전체 데이터를 로드하지 않고 'task' 컬럼만 읽으므로 매우 빠릅니다)
task_indices = defaultdict(list)
for idx, task in enumerate(tqdm(test_dataset['task'], desc="Grouping tasks")):
    task_indices[task].append(idx)

# 2. 각 Task별로 100개씩 랜덤 인덱스 추출
final_indices = []
seed = 42  # 재현성을 위한 시드 설정

for idx, (task, indices) in enumerate(task_indices.items()):
    # 데이터가 100개보다 적으면 있는 거 다 가져오고, 많으면 100개만 뽑음
    n_samples = min(len(indices), 100)
    sampled_indices = random.sample(indices, n_samples)
    final_indices.extend(sampled_indices)
    if idx > 4:
        break

# 3. 추출한 인덱스로 새로운 데이터셋 생성 (select는 메모리를 거의 안 씁니다)
# 마지막에 shuffle()을 해주는 것이 학습/테스트 시 편향을 막아줍니다.
temp_test_dataset = test_dataset.select(final_indices).shuffle(seed=seed)

print(f"Original Size: {len(test_dataset)}")
print(f"Sampled Size: {len(temp_test_dataset)}")

# 4. 저장하기
# 기존 경로 뒤에 '_sampled_100' 등을 붙여서 원본 덮어쓰기를 방지합니다.
original_path = "Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415"
save_path = original_path + "_sampled_100"

print(f"Saving to: {save_path}")
temp_test_dataset.save_to_disk(save_path)

Grouping tasks: 100%|██████████| 600/600 [00:00<00:00, 790135.76it/s]


Original Size: 600
Sampled Size: 600
Saving to: Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_sampled_100


Saving the dataset (1/1 shards): 100%|██████████| 600/600 [00:00<00:00, 21733.65 examples/s]


In [28]:
import numpy as np

np.unique(train_dataset['task'], return_counts=True)

(array(['bace', 'chebi-20-mol2text/0', 'chebi-20-text2mol/0',
        'forward_reaction_prediction', 'qm9_homo', 'qm9_homo_lumo_gap',
        'qm9_lumo', 'reagent_prediction', 'retrosynthesis',
        'smol-forward_synthesis/0', 'smol-molecule_captioning/0',
        'smol-molecule_generation/0', 'smol-name_conversion-i2s/0',
        'smol-name_conversion-s2i/0', 'smol-property_prediction-bbbp/0',
        'smol-property_prediction-clintox/0',
        'smol-property_prediction-esol/0',
        'smol-property_prediction-hiv/0',
        'smol-property_prediction-lipo/0',
        'smol-property_prediction-sider/0', 'smol-retrosynthesis/0'],
       dtype='<U34'),
 array([  1210,  25854,  25854, 121890, 117660, 117660, 117708, 121896,
        125565, 971351,  48531,  48532, 299890, 299890,   1569,   1144,
           888,  32863,   3360,  22820, 938530]))

In [ ]:
mol2text = test_dataset.filter(lambda x: x['task'] == 'chebi-20-mol2text/0')
for key, value in mol2text[0].items():
    print(key, ":",value)
    

x : [[5, 0, 4, 5, 3, 0, 2, 0, 0], [7, 0, 2, 5, 0, 0, 1, 0, 0], [5, 0, 3, 5, 0, 0, 1, 0, 0], [7, 0, 1, 5, 0, 0, 1, 0, 0], [5, 0, 3, 5, 1, 0, 1, 0, 0], [5, 0, 3, 5, 0, 0, 1, 0, 1], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 1, 4, 5, 1, 0, 2, 0, 1], [5, 2, 4, 5, 1, 0, 2, 0, 1], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 0, 3, 5, 0, 0, 1, 0, 1], [5, 0, 3, 5, 1, 0, 1, 0, 1], [5, 0, 3, 5, 0, 0, 1, 0, 1], [7, 0, 1, 5, 0, 0, 1, 0, 0], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 2, 4, 5, 0, 0, 2, 0, 1], [5, 0, 4, 5, 3, 0, 2, 0, 0], [5, 1, 4, 5, 1, 0, 2, 0, 1], [5, 0, 3, 5, 0, 0, 1, 0, 1], [7, 0, 1, 5, 0, 0, 1, 0, 0], [5, 0, 4, 5, 2, 0, 2, 0, 1], [5, 2, 4, 5, 0, 0, 2, 0, 1], [5, 0, 4, 5, 3, 0, 2, 0, 0]]
edge_index : [[0, 1, 1, 2, 2, 3, 2, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 14, 16, 16, 17, 17, 18, 18, 19, 18, 20, 20, 21, 21, 22, 21, 23, 23, 24, 24, 25, 24, 5, 24, 8, 20, 9, 18, 12], [1, 0, 2, 1, 3,

In [4]:
from datasets import load_from_disk

# dataset = load_from_disk("/appdataset/train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415")
chebi_20_mol2text_dataset = load_from_disk("Mol-LLM_Custom/dataset/real_train/chebi-20-mol2text_subtask-0_train")
forward_reaction_prediction_dataset = load_from_disk("Mol-LLM_Custom/dataset/real_train/forward_reaction_prediction_subtask-0_train")
custom_dataset = load_from_disk('Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415')
writer_dataset = load_from_disk('Mol-LLM_Custom/dataset/real_train(download_v1)/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer')
print(sorted(chebi_20_mol2text_dataset[0].keys()))
print(sorted(forward_reaction_prediction_dataset[0].keys()))
print(sorted(writer_dataset[0].keys()))
print(sorted(custom_dataset[0].keys()))


FileNotFoundError: Directory Mol-LLM_Custom/dataset/real_train/chebi-20-mol2text_subtask-0_train not found

In [14]:
print(sorted(chebi_20_mol2text_dataset[0].keys()))
print(sorted(forward_reaction_prediction_dataset[0].keys()))
print(sorted(writer_dataset[0].keys()))
print(sorted(custom_dataset[0].keys()))


['additional_edge_attr', 'additional_edge_index', 'additional_x', 'edge_attr', 'edge_index', 'input_mol_string', 'instruction', 'label', 'task_subtask_pair', 'x']
['additional_edge_attr', 'additional_edge_index', 'additional_x', 'edge_attr', 'edge_index', 'input_mol_string', 'instruction', 'label', 'task_subtask_pair', 'x']
['additional_edge_attr', 'additional_edge_index', 'additional_x', 'edge_attr', 'edge_index', 'input_mol_string', 'prompt_text', 'target_text', 'task', 'x']
['additional_edge_attr', 'additional_edge_index', 'additional_x', 'edge_attr', 'edge_index', 'input_mol_string', 'instruction', 'label', 'prompt_text', 'target_text', 'task', 'task_subtask_pair', 'x']


In [1]:
data = [0, 1]
data_0, data_1 = data

data_0, data_1

(0, 1)

In [6]:
from datasets import load_from_disk

writer_dataset = load_from_disk('Mol-LLM_Custom/dataset/real_train(download_v1)/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer')

writer_dataset

Dataset({
    features: ['task', 'x', 'edge_index', 'edge_attr', 'additional_x', 'additional_edge_index', 'additional_edge_attr', 'input_mol_string', 'prompt_text', 'target_text'],
    num_rows: 58757
})

In [7]:
import os
from datasets import load_from_disk, Dataset
from collections import defaultdict

# 1. 데이터셋 로드 (이미 하신 부분)
dataset_path = 'Mol-LLM_Custom/dataset/real_train(download_v1)/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_writer'
writer_dataset = load_from_disk(dataset_path)

# 저장할 경로 설정
save_path = 'Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_wrtier_100_sampled'

# 2. 태스크별로 100개씩 인덱스 추출 (효율적인 방법)
task_indices = defaultdict(list)
indices_to_select = []
samples_per_task = 100

print("Sampling data...")

# 전체 데이터셋의 'task' 컬럼만 순회하며 인덱스 수집
# (전체 로우를 로드하지 않아 메모리 효율적이고 빠름)
for idx, task in enumerate(writer_dataset['task']):
    # 해당 task가 아직 100개가 안 모였다면 인덱스 추가
    if len(task_indices[task]) < samples_per_task:
        task_indices[task].append(idx)
        indices_to_select.append(idx)

# 3. 추출된 결과 통계 출력
print("\n=== Sampled Counts per Task ===")
for task, indices in task_indices.items():
    print(f"{task}: {len(indices)}")

# 4. 선택된 인덱스로 새로운 데이터셋 생성
sampled_dataset = writer_dataset.select(indices_to_select)

# (선택 사항) 데이터가 task별로 뭉쳐있지 않게 섞기
sampled_dataset = sampled_dataset.shuffle(seed=42)

print(f"\nOriginal Size: {len(writer_dataset)}")
print(f"Sampled Size: {len(sampled_dataset)}")

# 5. 디스크에 저장
sampled_dataset.save_to_disk(save_path)
print(f"\nSaved sampled dataset to: {save_path}")

Sampling data...

=== Sampled Counts per Task ===
alchemy_homo: 100
alchemy_lumo: 100
alchemy_homo_lumo_gap: 100
aqsol-logS: 100
presto-forward_reaction_prediction: 100
presto-retrosynthesis: 100
smol-forward_synthesis: 100
smol-property_prediction-hiv: 100
smol-molecule_generation: 100
smol-retrosynthesis: 100
chebi-20-mol2text: 100
forward_reaction_prediction: 100
qm9_homo: 100
smol-property_prediction-sider: 100
smol-molecule_captioning: 100
smol-property_prediction-bbbp: 100
chebi-20-text2mol: 100
qm9_lumo: 100
reagent_prediction: 100
retrosynthesis: 100
smol-property_prediction-clintox: 100
smol-property_prediction-lipo: 100
bace: 100
smol-property_prediction-esol: 100
qm9_homo_lumo_gap: 100
orderly-forward_reaction_prediction: 100
orderly-retrosynthesis: 100

Original Size: 58757
Sampled Size: 2700


Saving the dataset (1/1 shards): 100%|██████████| 2700/2700 [00:00<00:00, 33494.49 examples/s]


Saved sampled dataset to: Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_wrtier_100_sampled


In [8]:
sampled_dataset[0]

{'task': 'alchemy_homo_lumo_gap',
 'x': [[5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 4, 5, 2, 0, 2, 0, 0],
  [5, 1, 4, 5, 0, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 1, 4, 5, 1, 0, 2, 0, 1],
  [5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 4, 5, 1, 0, 2, 0, 0],
  [5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 4, 5, 3, 0, 2, 0, 0]],
 'edge_index': [[0, 1, 1, 2, 2, 3, 2, 4, 2, 6, 3, 4, 4, 5, 6, 7, 6, 8],
  [1, 0, 2, 1, 3, 2, 4, 2, 6, 2, 4, 3, 5, 4, 7, 6, 8, 6]],
 'edge_attr': [[0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0],
  [0, 0, 0]],
 'additional_x': [[5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 4, 5, 2, 0, 2, 0, 0],
  [5, 1, 4, 5, 0, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 1, 4, 5, 1, 0, 2, 0, 1],
  [5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 4, 5, 1, 0, 2, 0, 0],
  [5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 4, 5, 3, 0, 2, 0, 0]],
 'addi

In [9]:
import re
import os

def parse_log_file(file_path):
    """
    로그 파일을 읽어서 Input Mol String을 키로 하는 딕셔너리를 반환합니다.
    멀티 GPU 환경에서 로그가 섞이더라도, 하나의 블록([DEBUG] ~ ----)이 
    연속적으로 출력되었다고 가정하고 정규표현식으로 추출합니다.
    """
    
    if not os.path.exists(file_path):
        print(f"Error: File not found - {file_path}")
        return {}

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # 정규표현식 패턴 정의
    # [DEBUG]로 시작해서 구분선(-----)으로 끝나는 패턴을 찾습니다.
    # re.DOTALL 옵션을 사용하여 줄바꿈(\n)도 문자로 인식하게 합니다.
    pattern = re.compile(
        r"\[DEBUG\]\s+Rank.*?Task:\s+(?P<task>.*?)\n"
        r".*?Input Mol String\s*:\s*(?P<input_mol>.*?)\n"
        r".*?Prompt \(Inference\):\s+(?P<prompt>.*?)\n"
        r".*?Target\s*:\s*(?P<target>.*?)\n"
        r".*?Prediction\s*:\s*(?P<prediction>.*?)\n"
        r".*?-{10,}", # 구분선 (---...)
        re.DOTALL
    )

    parsed_data = {}
    matches = pattern.finditer(content)
    
    count = 0
    for match in matches:
        count += 1
        data = match.groupdict()
        
        # SELFIES 문자열 앞뒤 공백 제거
        input_mol = data['input_mol'].strip()
        
        # 키(Input Mol String)가 중복될 경우, 가장 마지막 로그를 덮어쓰거나 리스트로 관리할 수 있습니다.
        # 여기서는 해당 샘플 정보를 저장합니다.
        parsed_data[input_mol] = {
            'task': data['task'].strip(),
            'target': data['target'].strip(),
            'prediction': data['prediction'].strip(),
            'raw_log_block': match.group(0) # 디버깅용 전체 텍스트
        }
    
    print(f"[{os.path.basename(file_path)}] Parsed {count} samples.")
    return parsed_data

def compare_logs(file_path1, file_path2):
    print(">>> Parsing Log Files...")
    data1 = parse_log_file(file_path1)
    data2 = parse_log_file(file_path2)
    
    # 두 데이터셋의 Input Mol String 교집합(공통 샘플) 찾기
    common_mols = set(data1.keys()) & set(data2.keys())
    
    print(f"\n>>> Comparison Result")
    print(f"File 1 Unique Samples: {len(data1)}")
    print(f"File 2 Unique Samples: {len(data2)}")
    print(f"Common Samples found: {len(common_mols)}\n")
    
    if len(common_mols) == 0:
        print("No common samples found. Check if the 'Input Mol String' is exactly the same in both files.")
        return

    # 공통 샘플에 대해 비교 출력
    for idx, mol_string in enumerate(common_mols):
        info1 = data1[mol_string]
        info2 = data2[mol_string]
        
        print(f"=== Common Sample #{idx+1} ===")
        print(f"Input Mol String: {mol_string[:100]}... (truncated)") # 너무 길면 잘라서 출력
        print(f"Task: {info1['task']}") # Task는 같다고 가정
        
        print(f"\n[File 1 Prediction]:")
        print(f"{info1['prediction']}")
        
        print(f"\n[File 2 Prediction]:")
        print(f"{info2['prediction']}")
        
        # Target은 보통 같아야 정상이지만 확인차 출력
        if info1['target'] != info2['target']:
            print(f"\n[Warning] Targets allow mismatch!")
            print(f"File 1 Target: {info1['target']}")
            print(f"File 2 Target: {info2['target']}")
        
        print("-" * 80)

# --- 실행 경로 설정 ---
path1 = 'temp_writier_100_sampled_with_print_total.txt'
path2 = 'temp_new_download_4_total.txt'

# 비교 함수 실행
compare_logs(path1, path2)

>>> Parsing Log Files...
[temp_writier_100_sampled_with_print_total.txt] Parsed 2613 samples.
[temp_new_download_4_total.txt] Parsed 2909 samples.

>>> Comparison Result
File 1 Unique Samples: 2174
File 2 Unique Samples: 2268
Common Samples found: 0

No common samples found. Check if the 'Input Mol String' is exactly the same in both files.


In [3]:
from datasets import load_dataset
smol_dataset = load_dataset(
    "osunlp/SMolInstruct",
    use_selfies=True,
    insert_core_tags=False,
    trust_remote_code=True,
    # ?download_mode="force_redownload"  # 이 옵션 추가
)

FileNotFoundError: datasets/osunlp/SMolInstruct@main/./data.zip

In [5]:
from datasets import load_from_disk

# dataset = load_from_disk("/appdataset/train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415")
download_dataset = load_from_disk('Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415')
writer_dataset = load_from_disk('Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_wrtier_100_sampled')
print(sorted(writer_dataset[0].keys()))
print(sorted(download_dataset[0].keys()))


['additional_edge_attr', 'additional_edge_index', 'additional_x', 'edge_attr', 'edge_index', 'input_mol_string', 'prompt_text', 'target_text', 'task', 'x']
['additional_edge_attr', 'additional_edge_index', 'additional_x', 'edge_attr', 'edge_index', 'input_mol_string', 'instruction', 'label', 'prompt_text', 'target_text', 'task', 'task_subtask_pair', 'x']


In [2]:
import torch
from torch.utils.data import DataLoader
from datasets import load_from_disk
from transformers import AutoTokenizer
import sys
import os

# 현재 작업 경로에 Mol-LLM_Custom 폴더가 있다고 가정하고 경로 추가
# (data_utils.py 등을 불러오기 위함)
sys.path.append(os.path.abspath("Mol-LLM_Custom"))

from data_utils import DataCollator
import model.added_tokens as added_tokens

# -----------------------------------------------------------------------------
# 1. 설정 (Configuration)
# -----------------------------------------------------------------------------
class MockArgs:
    """DataCollator에 필요한 인자들을 담은 Mock 객체"""
    def __init__(self):
        self.mol_representation = "string+graph" # "string+graph" 모드
        self.ignore_pad_token_for_loss = True
        self.train_molpo = False
        self.eval_molpo = False
        self.isomericSmiles = False
        self.canonical = False
        self.allHsExplicit = False
        self.projector_type = "qformer" # 또는 설정을 확인하여 "mlp" 등
        self.molpo_batch_division = 2 # 기본값
        self.selfies_enumeration = False
        # 필요한 경우 추가 인자 설정

args = MockArgs()

# -----------------------------------------------------------------------------
# 2. 토크나이저 로드 (Tokenizer)
# -----------------------------------------------------------------------------
# 사용하시는 LLM 모델 경로 (예: mistralai/Mistral-7B-Instruct-v0.3)
model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.3" 
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=False, padding_side="left")

# Special Token 추가 (학습 코드와 동일하게 맞춰야 함)
tokenizer.add_special_tokens({"pad_token": "<pad>"})
tokenizer.add_special_tokens({"eos_token": "</s>"})
tokenizer.add_special_tokens({"unk_token": "<unk>"})

# 추가 토큰 등록 (added_tokens.py 참조)
additional_tokens = [
    getattr(added_tokens, tokens)
    for tokens in dir(added_tokens)
    if not list(filter(lambda x: x.startswith("__"), [tokens]))
]
# 리스트 평탄화
flat_additional_tokens = []
for tok_list in additional_tokens:
    if isinstance(tok_list, list):
        flat_additional_tokens.extend(tok_list)
tokenizer.add_tokens(flat_additional_tokens)

# Mol Token 설정
tokenizer.add_special_tokens({"additional_special_tokens": ["<mol>"]})
tokenizer.mol_token_id = tokenizer.convert_tokens_to_ids("<mol>")

# -----------------------------------------------------------------------------
# 3. 데이터셋 로드 (Dataset Loading)
# -----------------------------------------------------------------------------
# 경로를 실제 데이터셋 경로로 수정해주세요.
original_data_path = "Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_wrtier_100_sampled"
custom_data_path   = "Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415" # 예시 경로

print(f"Loading Original Dataset from: {original_data_path}")
original_dataset = load_from_disk(original_data_path)

print(f"Loading Custom Dataset from:   {custom_data_path}")
custom_dataset = load_from_disk(custom_data_path)

# 비교를 위해 특정 Task만 필터링 (선택 사항)
target_task = "forward_reaction_prediction" # 또는 "bace" 등
print(f"\nFiltering for task: {target_task}")
orig_subset = original_dataset.filter(lambda x: target_task in x['task']).select(range(4)) # 4개만 샘플링
cust_subset = custom_dataset.filter(lambda x: target_task in x['task']).select(range(4))

# -----------------------------------------------------------------------------
# 4. DataLoader 생성 (DataCollator 사용)
# -----------------------------------------------------------------------------
collator = DataCollator(
    tokenizer=tokenizer,
    padding=True,
    max_length=512,
    return_tensors="pt",
    train=False, # Test 모드
    args=args
)

original_dataloader = DataLoader(orig_subset, batch_size=2, shuffle=False, collate_fn=collator)
custom_dataloader   = DataLoader(cust_subset, batch_size=2, shuffle=False, collate_fn=collator)

# -----------------------------------------------------------------------------
# 5. 배치 비교 및 출력 (Batch Inspection)
# -----------------------------------------------------------------------------
print("\nFetching batches...")
orig_batch = next(iter(original_dataloader))
cust_batch = next(iter(custom_dataloader))

def print_batch_details(batch, name="Batch"):
    print(f"\n{'='*20} {name} Details {'='*20}")
    
    # 1. Basic Keys
    print(f"Keys: {list(batch.keys())}")
    
    # 2. Text Input IDs & Decoded Text
    input_ids = batch['input_ids']
    print(f"\n[Input IDs Shape]: {input_ids.shape}")
    print(f"[Decoded Prompt (Sample 0)]:\n{tokenizer.decode(input_ids[0], skip_special_tokens=False)}")
    
    # 3. Labels (Target)
    labels = batch['labels']
    # -100 (Ignore index) 처리하여 디코딩
    valid_labels = labels[0].clone()
    valid_labels[valid_labels == -100] = tokenizer.pad_token_id
    print(f"\n[Decoded Label (Sample 0)]:\n{tokenizer.decode(valid_labels, skip_special_tokens=True)}")
    
    # 4. Graph Data (Main Graph)
    if 'graphs' in batch:
        graph = batch['graphs'] # Batch object
        # PyG Batch 객체를 리스트로 변환하여 첫 번째 그래프 접근
        g_list = graph.to_data_list()
        g0 = g_list[0]
        
        print(f"\n[Graph Data (Sample 0)]")
        print(f"  - Node Feature (x) Shape: {g0.x.shape}")
        print(f"  - Node Feature (x) Values (First 5 nodes):\n{g0.x[:5]}")
        print(f"  - Edge Index Shape: {g0.edge_index.shape}")
        print(f"  - Edge Attr Shape:  {g0.edge_attr.shape if g0.edge_attr is not None else 'None'}")
        
    # 5. Additional Graph Data (if exists)
    if 'additional_graphs' in batch:
        add_graph = batch['additional_graphs']
        ag_list = add_graph.to_data_list()
        ag0 = ag_list[0]
        print(f"\n[Additional Graph Data (Sample 0)]")
        print(f"  - Node Feature (x) Shape: {ag0.x.shape}")
        
    # 6. Raw Input Strings (if collator preserved them)
    if 'input_mol_strings' in batch:
        # DataCollator가 input_mol_strings를 토큰화해서 주는지, 리스트로 주는지 확인 필요
        # 보통 tokenizer.batch_decode를 해야 할 수도 있음
        if isinstance(batch['input_mol_strings'], torch.Tensor):
             print(f"\n[Raw Input Mol String (Decoded)]: {tokenizer.decode(batch['input_mol_strings'][0], skip_special_tokens=True)}")
        else:
             print(f"\n[Raw Input Mol String]: {batch['input_mol_strings'][0]}")

# 실행 및 출력
print_batch_details(orig_batch, "ORIGINAL")
print_batch_details(cust_batch, "CUSTOM")

# 직접 비교 (Graph X)
print("\n" + "="*50)
print("Direct Comparison: Graph Node Features (x)")
print("="*50)
orig_g0 = orig_batch['graphs'].to_data_list()[0]
cust_g0 = cust_batch['graphs'].to_data_list()[0]

print("Original Graph X[0]:\n", orig_g0.x)
print("Custom Graph X[0]:  \n", cust_g0.x)

if torch.equal(orig_g0.x, cust_g0.x):
    print("\n>> ✅ Graph Node Features are IDENTICAL.")
else:
    print("\n>> ❌ Graph Node Features are DIFFERENT.")

Loading Original Dataset from: Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_wrtier_100_sampled
Loading Custom Dataset from:   Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415

Filtering for task: forward_reaction_prediction


Filter: 100%|██████████| 152/152 [00:00<00:00, 1065.71 examples/s]


AttributeError: 'MockArgs' object has no attribute 'current_epoch'

In [2]:
from datasets import load_from_disk

# dataset = load_from_disk("/appdataset/train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415")
bace_test = load_from_disk('Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415')

In [4]:
bace_test[0]

{'x': [[5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [6, 0, 2, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [6, 0, 3, 5, 2, 0, 1, 0, 0],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 0],
  [5, 0, 4, 5, 1, 0, 2, 0, 0],
  [5, 0, 4, 5, 3, 0, 2, 0, 0],
  [5, 0, 3, 5, 0, 0, 1, 0, 0],
  [7, 0, 1, 5, 0, 0, 1, 0, 0],
  [6, 0, 3, 5, 1, 0, 1, 0, 0],
  [5, 0, 4, 5, 2, 0, 2, 0, 0],
  [5, 0, 4, 5, 1, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [7, 0, 2, 5, 0, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 0, 4, 5, 2, 0, 2, 0, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1],
  [5, 0, 3, 5, 0, 0, 1, 1, 1],
  [5, 0, 3, 5, 1, 0, 1, 1, 1]],
 'edge_index': [[0,
   1,
   1,
 

In [5]:
import re

def parse_log_file(file_path):
    """
    로그 파일을 읽어 샘플 단위로 분리하고, 
    {Prompt 내용: 전체 샘플 텍스트} 형태의 딕셔너리를 반환합니다.
    """
    data_map = {}
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            
        # 구분선(---...)을 기준으로 샘플 분리
        # 파일 내 구분선 길이에 따라 조절이 필요할 수 있으나, 보통 80개입니다.
        # 확실하게 나누기 위해 연속된 대시(-)가 20개 이상인 경우로 split 합니다.
        samples = re.split(r'-{20,}', content)
        
        for sample in samples:
            sample = sample.strip()
            if not sample:
                continue
                
            # Prompt (Inference) 부분 추출
            # "Prompt (Inference)" 부터 "Target" 전까지의 내용을 가져옵니다.
            prompt_match = re.search(r'Prompt \(Inference\)\s*:(.*?)Target\s*:', sample, re.DOTALL)
            
            if prompt_match:
                # 공백 제거 후 비교를 위해 정제
                prompt_content = prompt_match.group(1).strip()
                
                # 딕셔너리에 저장 (Key: Prompt, Value: 전체 Sample 텍스트)
                # 만약 중복된 Prompt가 파일 내에 여러 개라면 덮어씌워지므로, 
                # 필요시 리스트로 처리할 수 있습니다. (여기선 1:1 매칭 가정)
                data_map[prompt_content] = sample
                
    except FileNotFoundError:
        print(f"Error: 파일을 찾을 수 없습니다 -> {file_path}")
        return {}
        
    return data_map

# 1. 파일 경로 설정
file_path_1 = 'temp_writer_bace.txt'
file_path_2 = 'temp_download_bace_CC_INST.txt'

# 2. 각 파일 파싱
print("파일을 분석 중입니다...")
data1 = parse_log_file(file_path_1)
data2 = parse_log_file(file_path_2)

print(f"파일 1 샘플 수: {len(data1)}")
print(f"파일 2 샘플 수: {len(data2)}")
print("-" * 50)

# 3. 교집합 찾기 (공통된 Prompt 찾기)
common_prompts = set(data1.keys()) & set(data2.keys())

if common_prompts:
    print(f"!!! 동일한 Prompt를 가진 샘플 {len(common_prompts)}개를 발견했습니다 !!!\n")
    
    for i, prompt in enumerate(common_prompts, 1):
        print(f"=== [매칭된 샘플 Case #{i}] ===")
        print(f"\n>>> [File 1: temp_writer_bace.txt 에서의 내용]")
        print("-" * 80)
        print(data1[prompt])
        print("-" * 80)
        
        print(f"\n>>> [File 2: temp_download_bace_CC_INST.txt 에서의 내용]")
        print("-" * 80)
        print(data2[prompt])
        print("-" * 80)
        print("\n" + "="*80 + "\n")
else:
    print("두 파일 간에 Prompt가 동일한 샘플이 없습니다.")

파일을 분석 중입니다...
파일 1 샘플 수: 2663
파일 2 샘플 수: 150
--------------------------------------------------
!!! 동일한 Prompt를 가진 샘플 7개를 발견했습니다 !!!

=== [매칭된 샘플 Case #1] ===

>>> [File 1: temp_writer_bace.txt 에서의 내용]
--------------------------------------------------------------------------------
[DEBUG] Rank 1 | Batch 9 | Sample 0 | Task: bace
Input Mol String  : <SELFIES> [C] [S] [=Branch1] [C] [=O] [=Branch1] [C] [=O] [N] [Branch1] [=Branch2] [C] [=C] [C] [=C] [C] [=C] [Ring1] [=Branch1] [C] [=C] [C] [Branch2] [Ring2] [O] [C] [=Branch1] [C] [=O] [N] [C] [Branch1] [#Branch2] [C] [C] [=C] [C] [=C] [C] [=C] [Ring1] [=Branch1] [C] [Branch1] [C] [O] [C] [NH2+1] [C] [C] [=C] [C] [=C] [C] [Branch1] [=Branch2] [C] [Branch1] [C] [F] [Branch1] [C] [F] [F] [=C] [Ring1] [#Branch2] [=C] [C] [Branch1] [=Branch2] [N] [C] [C] [C] [C] [Ring1] [Branch1] [=O] [=C] [Ring2] [Ring2] [=Branch1] </SELFIES> 
--- Graph Data ---
  x (shape=[49, 9]): [[5, 0, 4, 5, 3, 0, 2, 0, 0], [15, 0, 4, 5, 0, 0, 2, 0, 0], [7, 0, 1, 5, 0

In [6]:
from datasets import load_from_disk

test_dataset = load_from_disk("Mol-LLM_Custom/dataset/real_train/mistralai-Mistral-7B-Instruct-v0.3_string+graph_q32_test_3.3M_0415_download")

In [8]:
len(test_dataset[0])

13